
### PageRank

In this file I implement PageRank using OOP and iterators on [The Hamilton Project]'s dataset on **who talks about whom** in each of the show's songs. When character A mentions character B, we'll think of this as a *link* from A to B, suggesting that B might be important.

First to setup, I ran these functions provided to me on the assignment. Their purpose is to get the data, save to local computer, read in the data, and return a list of tuples. 

In [16]:

import urllib
import csv

def retrieve_data(url):
    """
    Retrieve a file from the specified url and save it in a local file 
    called data.csv. The intended values of url are: 
    
    1. https://philchodrow.github.io/PIC16A/homework/HW3-hamilton-data.csv
    2. https://philchodrow.github.io/PIC16A/homework/HW3-flights-data.csv
    """
    
    # grab the data and parse it
    filedata = urllib.request.urlopen(url) 
    to_write = filedata.read()
    
    # write to file
    with open("data.csv", "wb") as f:
        f.write(to_write)
        
def read_data(path):
    """
    read downloaded data from a .csv file, and return a list of tuples. 
    each tuple represents a link between states. 
    """
    with open(path, "r") as f:
        reader = csv.reader(f)
        return [(row[0], row[1]) for row in list(reader)]

Next I get the hamilton data.

In [17]:

url = "https://philchodrow.github.io/PIC16A/homework/HW3-hamilton-data.csv"
retrieve_data(url) 
data=read_data("data.csv")

Next I wrote a function to describe the structure of the data

In [18]:
#capitalize
def describe(n):
    mentioner=(str(data[n][0])).capitalize()
    mentionee=(str(data[n][1])).capitalize()
    return ("Element " + str(n) + " of the Hamilton data set is " + str(data[n]) + ". That means " + mentioner + " mentions " + mentionee + " in a song.")

describe(5)

"Element 5 of the Hamilton data set is ('burr', 'betsy'). That means Burr mentions Betsy in a song."

Next I wrote a function called that converts the data into a dictionary.

In [19]:
def data_to_dictionary(data):
    '''
    This function takes in a list of tiples and returns a dictionary containing each of the unique mentioners as the keys
    and a list of all mentionees paired with the first parts as the vals
    param-data, data list of mentioners and mentionees
    return- dictionary of mentioners and their mentionees 
    '''
    dic={}
    for key,val in data: #for each pair in data ex: (a, b)
        dic.setdefault(key, []).append(val) #set the default key to a, default val to blank list, add occurances of the vals to the list
    return dic
 

Next I implemented a DiGraph class.

In [20]:
class PR_DiGraph:
    def __init__(self, data, iteration_limit):
        '''
        initializer method- sets initial values
        param-self, data(list of mentioner data), iteration limit(max amt of iterations)
        '''
        self.link_dict=data_to_dictionary(data)
        self.iteration_limit=iteration_limit
        
    def linked_by(self,x):
        '''
        returns the linked charachter at the name index
        param- self, x(index which is a name)
        return- the linked charachter at the given name index
        
        #this returns a list of states linked from x
        '''
        return self.link_dict[x] 
    
    def __iter__(self):
        '''
        iterator method that returns object of PR_Iterator  
        '''
        return (PR_iterator(self)) 

In [21]:
D = PR_DiGraph(data, iteration_limit = 10000)
D.linked_by('peggy')

['peggy', 'schuylerSis']

Next I implemented a PR_iterator class that has an initializer and functions to represent each type of movement along the data: follow link, next, and teleport. 

In [22]:
import random

class PR_iterator:
    
    def __init__(self, PR_d):
        '''
        Initializer method-sets values
        param- self and PR_d, an object of PR_di class
        '''
        self.PR_d=PR_d
        self.i=0
        self.current_state = "hamilton" 
        self.iteration_limit=PR_d.iteration_limit #he didnt have this part
        
    def follow_link(self):
        '''
        This function picks where to go from the current state 
        it will go from the current mention a random charchter they mention, if it selects a valid next charachter link,
        follow that link, otherwise teleport to a random chrachter mention
        
        param-self
        '''

        try: #the reason we need this is because some charachters dont talk
            
            #** this is a list of possibilities actually
            state=self.PR_d.linked_by(self.current_state) #** wrong-get the linked by chrachter
            
            next_state =random.choice(state) #go from linked by charachter to random next charachter
        
        
            if self.current_state != next_state: #if they are not same charachter, set current to next
                self.current_state=next_state
            else: #if they are same charachter, time to teleport
                self.teleport()
        except KeyError:
            self.teleport()
            
       
    def __next__(self):
        '''
        Next method- increments counter, checks if counter is within limit, 
        chooses next state with 85% prb of following a link, 15% prb of random teleportation
        param-self
        return- the current state
        '''
        self.i+=1 #increment counter
        if self.i > self.iteration_limit: #make sure counter is within iteration limit, if not raise error
            raise StopIteration
            
        if random.random() < 0.85:  
            self.follow_link() #remember when ur using the functions of the iterator class
        else:
            self.teleport()
        
        return self.current_state
        
        
        
    def teleport(self):
        """
        This function chooses a random next state
        param-self
        """
        options_for_next=list(self.PR_d.link_dict.keys())
        self.current_state = random.choice(options_for_next)
        #another way is self.current_state=random.couce(list(self.PR_d.link_dict.jeys())), one line
        #
            
            
    

In [24]:
# run the below
D = PR_DiGraph(data, iteration_limit = 5)
for char in D:
    print(char)

reynolds
angelica
hamilton
schuylerSis
laurens


Using the above classes, I ran PageRank to find which charachters were the most relevant.

In [26]:

pr_score_dict={} #start with empty dict
pRank=PR_DiGraph(data, 1000000) #create page rank

for state in pRank: #create a dictionary that keeps track of frequency of each state, key=state (charachter),val=frequency
    if state in pr_score_dict: #if charachter already mentioned
        pr_score_dict[state] +=1  #add to count
    else:
        pr_score_dict[state] = 1 #otherwise, start count at 1
        

Then I displayed the top 10 most relevant charachters. 

In [27]:
#print the things in descending order based on value
list_of_scores=[(keys,vals) for keys,vals in pr_score_dict.items()] #make list from key and val
sorted((list_of_scores),key= lambda list_of_scores: list_of_scores[1], reverse=True)[:10] #sort in reverse order and print indecies 0-9

[('hamilton', 166579),
 ('burr', 99146),
 ('washington', 92523),
 ('jefferson', 72298),
 ('eliza', 51955),
 ('angelica', 47672),
 ('madison', 36658),
 ('lafayette', 34230),
 ('lee', 33572),
 ('jAdams', 30964)]